In [ ]:
import pandas as pd

In [ ]:
test_path = "/home/monlamai/Documents/GitHub/combining-dataset/550-train.tsv"
df = pd.read_csv(test_path, sep='\t')

In [ ]:
df.head()

In [ ]:
df = df[df['dept'] == 'STT_TT']
df['len'] = df['uni'].apply(lambda x: len(x))

In [ ]:
df.head()

In [ ]:
df.sort_values(by=['len'], inplace=True, ascending=False, ignore_index=True)

In [117]:
# 60005
# 70021
i = len(df) - 70021
df.loc[i, 'url'], df.loc[i, 'uni']

('https://d38pmlk0v88drf.cloudfront.net/wav/STT_TT00015_05751.750-05752.550.wav',
 'བསྔགས་པ་ཕྱོགས་བཞིར་ ')

In [118]:
from transformers import pipeline
import pyewts
generator = pipeline(model="openpecha/wav2vec2_run8")

opt = generator(df.loc[i, 'url'])
inf = opt['text']
print(inf)

generator = pipeline(model="TenzinGayche/whisper-small-3")
opt = generator(df.loc[i, 'url'])
inf = opt['text']

converter = pyewts.pyewts()
print(converter.toUnicode(inf))

བསྔགས་པ་མཆོག་ཞིར་
སྔགས་པ་ཕྱོགས་བཞིར་


In [ ]:
# import os
from transformers import pipeline
from datasets import Dataset

generator = pipeline(model="openpecha/wav2vec2_run8", device="cuda:0")


# opt = generator(df.loc[0, 'url'])
# inf = opt['text']
# print(inf)

dataset = Dataset.from_pandas(df)

def generate(batch):
    urls = batch["url"]
    generated_texts = generator(urls)  # This should process all URLs in the batch at once
    # Extracting the 'text' field from each generated output and assigning it back to the batch
    batch['inf'] = [item['text'] for item in generated_texts]
    return batch

# df['inf']  = df.apply(lambda row: generator(row['url'])['text'], axis=1)
# updated_dataset = dataset.map(generate, batched=True, batch_size=8)

In [ ]:
df_out = updated_dataset.to_pandas()

In [ ]:
# import pyewts

# converter = pyewts.pyewts()

# df_out['inf_uni'] = df_out['inf'].map(lambda x : converter.toUnicode(x))

In [ ]:
from evaluate import load

cer_metric = load("cer")

In [ ]:
df_out

cer = cer_metric.compute(references=[df_out.loc[0, 'uni']], predictions=[df_out.loc[0, 'inf']])

print(cer)

In [ ]:
df_out['cer'] = df_out.apply(lambda row: min(cer_metric.compute(references=[df_out.loc[0, 'uni']], predictions=[df_out.loc[0, 'inf']]), 1), axis=1)

In [ ]:
mean_cer = df_out['cer'].mean()

In [ ]:
mean_cer

In [ ]:
mean_cer * 100

In [ ]:
df_out.head()

In [ ]:

df_out.to_csv('test_wav2vec2.tsv', sep='\t', index=False)